In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Базовая работа с изображением

In [ ]:
image = cv2.imread('sar_2_color.jpg')

In [ ]:
plt.imshow(image)

In [ ]:
image.shape  # h,w,c

In [ ]:
image[250, 250]  # b,g,r

In [ ]:
# ROI
img_roi = image[100:200, 500:700]

In [ ]:
plt.imshow(img_roi)

In [ ]:
b, g, r = cv2.split(image)

In [ ]:
plt.imshow(b, cmap='gray')

In [ ]:
plt.imshow(g, cmap='gray')

In [ ]:
# alternative approach
b = image[:, :, 0]

In [ ]:
import copy

image2 = copy.deepcopy(image)

In [ ]:
image2[50:100, 50:100] = [0, 0, 0]

In [ ]:
plt.imshow(image2)

In [ ]:
# empty image
image_template = np.zeros(image.shape, np.uint8)

In [ ]:
plt.imshow(image_template)

# Конвертация цветовых моделей

In [ ]:
image_template[0, 0]

In [ ]:
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [ ]:
image_gray[0, 0]

In [ ]:
image_gray.shape

In [ ]:
image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

In [ ]:
image_hsv.shape

In [ ]:
image_hsv[0, 0]

In [ ]:
image[0, 0]

In [ ]:
image_lab = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)

In [ ]:
image_lab[0, 0]

# Пороговая фильтрация

In [ ]:
_, thresh1 = cv2.threshold(image_gray, 200, 255, cv2.THRESH_BINARY)

In [ ]:
plt.imshow(thresh1, cmap='gray')

In [ ]:
thresh1[thresh1 == 100].sum()

# Построение гистограммы

In [ ]:
histSize = 256
histRange = (0, 256)
accumulate = False

b_hist = cv2.calcHist([b], [0], None, [histSize], histRange, accumulate=accumulate)

In [ ]:
plt.plot(b_hist)

In [ ]:
b_hist_cum = b_hist.cumsum()

In [ ]:
plt.plot(b_hist_cum)

In [ ]:
b_hist_norm = b_hist / (image.shape[0] * image.shape[1])

In [ ]:
plt.plot(b_hist_norm)

# Сравнение двух изображений

In [ ]:
from skimage.metrics import structural_similarity, mean_squared_error

(ssim, diff) = structural_similarity(image_gray, image_gray, full=True)
diff = (diff * 255).astype("uint8")
print("SSIM: {}".format(ssim))

In [ ]:
plt.imshow(diff)

In [ ]:
mse = mean_squared_error(image_gray, image_gray)
print(mse)
mse

# Статистические характеристики изображений

In [ ]:
mean = image_gray.mean()

In [ ]:
std = image_gray.std()

In [ ]:
print(mean, std)

In [ ]:
eq_gray = cv2.equalizeHist(image_gray)

In [ ]:
plt.imshow(eq_gray, cmap="gray")


In [ ]:
plt.imshow(image_gray, cmap="gray")

In [ ]:
# 1. Загрузите изображение в оттенках серого sar_1_gray.jpg.
# 2. постройте гистограмму
# 3. реализуйте алгоритм гамма коррекции с параметром гамма <1, >1.
# 4. Сравните исходное изображение, скорректированное при помощи гамма-фильтра. MSE, SSIM.
# 5. реализуйте алгоритм статистической цветокоррекции на основе статистики eq_gray.
# 6. Протестируйте работу алгоритмов пороговой фильтрации с различными параметрами.
# Для каждого решения - напечатайте результат


# Lab Work

In [ ]:
image = cv2.imread('sar_1_gray.jpg')

In [ ]:
plt.imshow(image)

In [ ]:
b = image[:, :, 0]

In [ ]:
histSize = 256
histRange = (0, 256)
accumulate = False

b_hist = cv2.calcHist([b], [0], None, [histSize], histRange, accumulate=accumulate)

In [ ]:
plt.plot(b_hist)

In [ ]:
def gamma_correction(source, gamma):
    # Нормализуем значения пикселей к диапазону [0, 1]
    normalized = source / 255.0

    # Применяем гамма-коррекцию
    corrected = np.power(normalized, gamma)

    # Вовращаем в диапазон [0, 255]
    result = (corrected * 255).astype(np.uint8)

    return result

In [ ]:
# Гамма < 1 (осветление)
gamma_low = 0.5
image_gamma_low = gamma_correction(image, gamma_low)
plt.imshow(image_gamma_low)

In [ ]:
# Гамма > 1
gamma_high = 2.0
image_gamma_high = gamma_correction(image, gamma_high)
plt.imshow(image_gamma_high)

In [ ]:
# gamma_low
(ssim_low, diff_low) = structural_similarity(image, image_gamma_low, full=True, win_size=3)
diff_low = (diff_low * 255).astype("uint8")
mse_low = mean_squared_error(image, image_gamma_low)
print("SSIM: {}".format(ssim_low))
print(f"MSE: {mse_low:.2f}")
plt.imshow(diff_low)

In [ ]:
# gamma_high
(ssim_high, diff_high) = structural_similarity(image, image_gamma_high, full=True, win_size=3)
diff_high = (diff_high * 255).astype("uint8")
mse_high = mean_squared_error(image, image_gamma_high)
print("SSIM: {}".format(ssim_high))
print(f"MSE: {mse_high:.2f}")
plt.imshow(diff_high)

In [ ]:
def statistical_color_correction(source):
    source_gray = cv2.cvtColor(source, cv2.COLOR_BGR2GRAY)
    eq_gray = cv2.equalizeHist(source_gray)

    source_mean = source.mean()
    source_std = source.std()

    eq_gray_mean = eq_gray.mean()
    eq_gray_std = eq_gray.std()

    corrected = ((source.astype(np.float32) - source_mean) * (eq_gray_std / source_std) + eq_gray_mean)

    corrected = np.clip(corrected, 0, 255).astype(np.uint8)

    return corrected

In [ ]:
image_stat_corrected = statistical_color_correction(image)

fig, axes = plt.subplots(1, 2, figsize=(15, 5))
axes[0].imshow(image)
axes[1].imshow(image_stat_corrected)

In [ ]:
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

thresholds = [50, 100, 150, 200]

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('Простая бинарная фильтрация', fontsize=16)

for idx, thresh_val in enumerate(thresholds):
    _, thresh_result = cv2.threshold(image_gray, thresh_val, 255, cv2.THRESH_BINARY)
    row = idx // 2
    col = idx % 2
    axes[row, col].imshow(thresh_result)
    axes[row, col].set_title(f'Порог = {thresh_val}')
    axes[row, col].axis('off')